# 🚀 OpenSloth Demo Training Notebook

This notebook demonstrates how to fine-tune large language models using opensloth's multi-GPU capabilities. It's equivalent to running:

```bash
opensloth-train examples/example_sharegpt_lora_2gpus.py
```

## What This Demo Does

- **Multi-GPU Training**: Uses 2 GPUs with NCCL synchronization
- **Adaptive Batching**: Optimizes sequence sorting and padding
- **LoRA Fine-tuning**: Efficient parameter updates with Low-Rank Adaptation
- **Response-only Loss**: Calculates loss only on assistant responses

## Prerequisites

1. opensloth installed: `pip install git+https://github.com/anhvth/opensloth.git`
2. At least 2 GPUs available (adjust `gpus=[0, 1]` if needed)
3. Sufficient VRAM (reduce batch size if needed)

In [6]:
pwd

'/Users/anhvth/projects/opensloth/notebooks'

In [7]:
%%capture
%load_ext autoreload
%autoreload 2
%cd ../

In [8]:
!pip install -e ./

Obtaining file:///Users/anhvth/projects/opensloth
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached fastcore-1.8.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Using cached jupyterlab-4.4.3-py3-none-any.whl.metadata (16 kB)
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.16.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any

In [12]:
# Import opensloth configuration classes
from opensloth.opensloth_config import *

# Check GPU availability
import torch
print(f'🔥 CUDA Available: {torch.cuda.is_available()}')
print(f'🔥 GPU Count: {torch.cuda.device_count()}')
for i in range(torch.cuda.device_count()):
    print(f'   GPU {i}: {torch.cuda.get_device_name(i)}')


ModuleNotFoundError: No module named 'opensloth'

## ⚙️ Configuration Setup

HyperSloth uses Pydantic models for type-safe configuration. We'll set up:

1. **Data Configuration**: Dataset and tokenization settings
2. **Training Configuration**: GPU allocation and loss calculation
3. **Model Configuration**: Base model and LoRA parameters
4. **Training Arguments**: Learning rate, batch size, and optimization settings

In [ ]:
from opensloth.opensloth_config import *
from opensloth.scripts.hp_trainer import run_mp_training, setup_envs

# Main configuration using Pydantic models
opensloth_config = OpenSlothConfig(
    data=HFDatasetConfig(
        dataset_name="mlabonne/FineTome-100k",
        split="train",
        tokenizer_name="Qwen/Qwen3-8B",  # does not matter same family qwen3
        num_samples=5000,
        instruction_part="<|im_start|>user\n",
        response_part="<|im_start|>assistant\n",
        chat_template="qwen3",
    ),
    training=TrainingConfig(
        gpus=[0, 1, 2, 3],
        loss_type="response_only",
    ),
    fast_model_args=FastModelArgs(
        model_name="unsloth/Qwen3-8B-bnb-4bit",
        max_seq_length=32000,
        load_in_4bit=True,
    ),
    lora_args=LoraArgs(
        r=8,
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_dropout=0,
        bias="none",
        use_rslora=False,
    ),
)

# Training arguments using Pydantic model
training_config = TrainingArgsConfig(
    output_dir="outputs/qwen3-8b-FineTome-4gpus/",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    logging_steps=3,
    num_train_epochs=3,
    lr_scheduler_type="linear",
    warmup_steps=5,
    save_total_limit=2,
    weight_decay=0.01,
    optim="adamw_8bit",
    seed=3407,
    report_to="tensorboard",  # tensorboard or wawndb
)

setup_envs(opensloth_config, training_config)

run_mp_training(
    opensloth_config.training.gpus, opensloth_config, training_config
)

Global batch size: 32
[MP] Running on 4 GPUs


07:33:23 | INFO     | GPU2 | hp_trainer.py:42 | Training on GPU 2 with output_dir outputs/qwen3-8b-FineTome-4gpus/
07:33:23 | INFO     | GPU2 | hp_trainer.py:45 | 🚀 Starting total training timer
07:33:23 | INFO     | GPU3 | hp_trainer.py:42 | Training on GPU 3 with output_dir outputs/qwen3-8b-FineTome-4gpus/
07:33:23 | INFO     | GPU3 | hp_trainer.py:45 | 🚀 Starting total training timer
07:33:23 | INFO     | GPU0 | hp_trainer.py:42 | Training on GPU 0 with output_dir outputs/qwen3-8b-FineTome-4gpus/
07:33:23 | INFO     | GPU0 | hp_trainer.py:45 | 🚀 Starting total training timer
07:33:23 | INFO     | GPU1 | hp_trainer.py:42 | Training on GPU 1 with output_dir outputs/qwen3-8b-FineTome-4gpus/
07:33:23 | INFO     | GPU1 | hp_trainer.py:45 | 🚀 Starting total training timer
